# Data API - Fetch and append to S3 object

In [ ]:
pip install boto3

In [1]:
import requests
import boto3
import pandas as pd
import io

In [ ]:
# Columns from your dataset
columns = [
    "Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
    "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points", "Wilderness_Area",
    "Soil_Type", "Cover_Type"
]

# S3/MinIO config
s3 = boto3.client(
    "s3",
    endpoint_url="http://minio:9000",
    aws_access_key_id="admin",
    aws_secret_access_key="supersecret",
    region_name="us-east-1"
)
bucket_name = "covertype-data"
object_key = "responses.csv"

# Ensure bucket exists
try:
    s3.head_bucket(Bucket=bucket_name)
except:
    s3.create_bucket(Bucket=bucket_name)

def fetch_and_append(group_number=1):
    # Fetch data from FastAPI
    url = "http://api:80/data"
    params = {"group_number": group_number}
    response = requests.get(url, params=params)
    data = response.json()["data"]

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=columns)

    # Download existing CSV from S3 if exists
    try:
        obj = s3.get_object(Bucket=bucket_name, Key=object_key)
        existing_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
        df = pd.concat([existing_df, df], ignore_index=True)
    except s3.exceptions.NoSuchKey:
        pass

    # Save back to S3
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucket_name, Key=object_key, Body=csv_buffer.getvalue())
    print(f"Appended data for group {group_number} to S3.")

# Example usage:
fetch_and_append(group_number=10)